In [1]:
with open("RegulonDB_TF.txt", "r") as R:
    TFB_raw = R.read()
    
with open("RegulonDB_Promoter.txt", "r") as R:
    Promoters_raw = R.read()

In [2]:
def split_by_upper(z): 
    for i in range(len(z)):
        if z[i] == z[i].upper():
            return z[:i]

In [3]:
TFB = TFB_raw.split("\n")
Promoters = Promoters_raw.split("\n")

Stripped_TFB = []
for x in TFB:
    if len(x) > 0:
        if x[0] != "#":
            Stripped_TFB.append(x.split("\t"))
        
Stripped_Promoters = []
for x in Promoters:
    if len(x) > 0:
        if x[0] != "#":
            Stripped_Promoters.append(x.split("\t"))


In [4]:
CRP_TFB = []
for x in Stripped_TFB:
    if x[1] == "CRP":
        CRP_TFB.append(x)

CRP_Promoters = []
for x in CRP_TFB: # Ow, O(n^2). But, it doesn't seem to matter too much
    for y in Stripped_Promoters:
        if x[9] == y[1]:
            CRP_Promoters.append(y)


In [5]:
def get_uppercase_chars(s):
    build = []
    for i in range(len(s)):
        if s[i].upper() == s[i]:
            build.append(s[i])
    return "".join(build)

In [6]:
pos_promoters_is = []
for i, x in enumerate(CRP_TFB):
    if x[8] == "+" #and x[-1] == "Strong":
        pos_promoters_is.append(i)

seqs = []
for i in pos_promoters_is:
    #dna = get_uppercase_chars(CRP_TFB[i][11]) 
    dna = split_by_upper(CRP_Promoters[i][5]).upper() #split by upper?
    seqs.append(dna)


new_seqs = [] 
max_length_seqs = max([len(x) for x in seqs])
for i, x in enumerate(seqs):
    if len(x) != 0:
        new_seqs.append(x + "-"*(max_length_seqs - len(x)))
    else:
        pos_promoters_is.remove(pos_promoters_is[i])
seqs = new_seqs


In [7]:
from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna
from Bio import AlignIO
seqrecords = [SeqRecord(Seq(z, generic_dna), id = str(pos_promoters_is[i])) for i, z in enumerate(seqs)]
unaligned = MultipleSeqAlignment(seqrecords)
AlignIO.write(unaligned, "zz.fasta", "fasta")

1

In [8]:
from Bio.Align.Applications import MuscleCommandline
muscle_exe = "muscle.exe"
muscle_cline = MuscleCommandline(muscle_exe, input="zz.fasta", out = "res.fasta")
stdout, stderr = muscle_cline()


In [9]:
from Bio.Align.AlignInfo import SummaryInfo
aligned = AlignIO.read("res.fasta", "fasta")
print(SummaryInfo(aligned).gap_consensus(0.5))

----------------------------XXXXXXXXXXXXXXXXXXXXXXXXTTXXXXXTX----XXXXXXXXXXXXXXT------------XXXXXXX----------------------------


In [21]:
aligned_seqs = [z.seq for z in aligned]
ATCG = [[0 for y in range(5)] for x in range(len(aligned_seqs[0]))]
options = ["A", "T", "C", "G", "-"]
for seq in aligned_seqs:
    for i in range(len(seq)):
        for io, base in enumerate(options):
            if seq[i] == base:
                ATCG[i][io] +=1
                break

consensus = []
for point in ATCG:
    actual_total = sum(point[0:4])
    proportions = [base/actual_total for base in point[0:4]]
    maxN = max(proportions)
    n = 8
    if maxN > 0.40 and actual_total > n:
        consensus.append(options[proportions.index(maxN)])
    elif actual_total < n:
        consensus.append("-")
    else:
        consensus.append("X")
            
print("".join(consensus))




-----------XATGTXXXXXXXXTXXXXXXXAXXAXXXXXTXXAXXXXTXXTTTXAXXTT----TXXTXXXXTAAXTTT----XXXXTXXXXTAXXXTXXTATTXXTTXCXXCC------------
103
102
101
100
91
83
80
74
71
66
63
62
61
59
56
55
50
47
42
41
40
38
37
36
27
22
21
17
14
13
12
11
8
7
5
2
1
